In [1]:
import numpy

In [3]:
import pandas as pd
import folium
from folium.plugins import HeatMap


In [4]:
# Details of API at:- https://aqicn.org/api/
base_url = "https://api.waqi.info"

# Storing a special User token
tok = "6cec9fff29506692c24dcb2a603e5561461da99d" # Token available at :- https://aqicn.org/data-platform/token/#/

# Storing latitude and longitude in format: (lat, long)-> Bottom Left ; (lat, lon)-> Top Right
latlngbox = "8,61,37,97" # Location of India is 8N 61E to 37N, 97E approx
trail_url=f"/map/bounds/?latlng={latlngbox}&token={tok}"

# Calling data from URL
my_data = pd.read_json(base_url + trail_url)
my_data

,status,data
0,ok,"{'lat': 11.9309, 'lon': 79.8027, 'uid': 13698,..."
1,ok,"{'lat': 27.64665, 'lon': 85.3234, 'uid': 14868..."
2,ok,"{'lat': 32.5, 'lon': 80.1161, 'uid': 7069, 'aq..."
3,ok,"{'lat': 27.554793, 'lon': 76.611536, 'uid': 11..."
4,ok,"{'lat': 30.1480566, 'lon': 77.2893471, 'uid': ..."
...,...,...
199,ok,"{'lat': 11.4068288, 'lon': 76.7138973, 'uid': ..."
200,ok,"{'lat': 13.008556, 'lon': 76.125258, 'uid': 13..."
201,ok,"{'lat': 20.362421, 'lon': 72.918013, 'uid': 12..."
202,ok,"{'lat': 24.58434363, 'lon': 80.8549414, 'uid':..."


In [5]:
all_rows = []
for each_row in my_data['data']:
    all_rows.append([each_row['station']['name'],each_row['lat'],each_row['lon'],each_row['aqi']])
df = pd.DataFrame(all_rows, columns=['station_name', 'lat', 'lon', 'aqi'])
print(df)


,station_name,lat,lon,aqi
0,"Jawahar Nagar, Puducherry, India",11.930900,79.802700,31
1,"Khumaltar, Nepal",27.646650,85.323400,77
2,"Ngari Station, Ngari Sanai (阿里地区阿里监测站)",32.500000,80.116100,47
3,"Moti Doongri, Alwar, India",27.554793,76.611536,79
4,"Gobind Pura, Yamuna Nagar, India",30.148057,77.289347,78
...,...,...,...,...
199,"Bombay Castel, Ooty, India",11.406829,76.713897,93
200,"B.Katihalli, Hassan, India",13.008556,76.125258,104
201,"Phase-1 GIDC, Vapi, India",20.362421,72.918013,139
202,"Bandhavgar Colony, Satna, India",24.584344,80.854941,122


#Visualization Of Live_HeatMap using folium 

In [6]:

df['aqi'] = pd.to_numeric(df.aqi, errors='coerce')
# Removing NaN Values in AQIs Column
df1 = df.dropna(subset = ['aqi'])

In [7]:
df2 = df1[['lat', 'lon', 'aqi']] # Second Data Frame without State Name
init_loc = [22, 80] # Approx location info of Centre of India to set as centre of map

m = folium.Map(location = init_loc, zoom_start = 5)
heat_aqi = HeatMap(df2, radius = 15, blur = 10)
m.add_child(heat_aqi)
m # Show the map

Visualization Of Various Stations on Map using folium

In [8]:
m2 = folium.Map(location = init_loc, zoom_start= 5)
for _, row in df1.iterrows():
    lat = row['lat']
    lon = row['lon']
    station = row['station_name'] + ' AQI=' + str(row['aqi'])
    station_aqi = row['aqi']
    if station_aqi > 300: ## Black for very bad AQI
        pop_color = 'black'
    elif station_aqi > 200:
        pop_color = 'darkred' ## Red for moderate AQI
    else:
        pop_color = 'cadetblue' ## Blue for good AQI
    folium.Marker(location= [lat, lon],popup = station,icon = folium.Icon(color = pop_color)).add_to(m2)
m2 # Display map
